#計算機程式設計二
#第十一週上課內容
## Constructors


## 自己定 List

這個範例的程式碼稍微多一點，用到了 template、smart pointers、lambda functions。

實作 List 常用的功能，例如 map、fold (reduce)、filter 等等。

In [ ]:
%%writefile E10_02.cpp
#include <iostream>
#include <memory>
#include <cstdlib>


using namespace std;
template<typename T>
class List {
private:
    struct Node
    {
        T val;
        shared_ptr<const Node> next;
        Node(T v, const shared_ptr<const Node>& n): val{v}, next{n}  { }
    };

    shared_ptr<const Node> head;

    explicit List (const shared_ptr<const Node> & nodes) : head{nodes} { }

public:

    List() { } //head{nullptr} done by shared_ptr

    List(T val, const List & tail): head{make_shared<Node>(val, tail.head)} { }

    bool isEmpty() const { return (head == nullptr); }

    T car() const {
        if (isEmpty()) exit(EXIT_FAILURE);
        return head->val;
    }
    List cdr() const {
        if (isEmpty()) exit(EXIT_FAILURE);
        return List(head->next);
    }

    void show() const {
        if (isEmpty()) std::cout << ".\n";
        else {
            std::cout << car() << "->";
            cdr().show();
        }
    }
};


template <typename T>
List<T> cons(T val, List<T> tail)
{
    return List<T>{val, tail};
}

template<typename S, typename T, typename F>
List<S> mapl(F f, List<T> lst)
{
    if (lst.isEmpty())
        return List<S>();
    else
        return List<S>(f(lst.car()), mapl<S>(f, lst.cdr()));
}

template<typename T, typename P>
List<T> filter(P p, List<T> lst)
{
    if (lst.isEmpty())
        return List<T>();
    else {
      if (p(lst.car())) {
        return List<T>(lst.car(), filter(p, lst.cdr()));
      } else {
        return filter(p, lst.cdr());
      }
    }
}

template<typename S, typename T, typename F>
S foldr(F f, S accumul, List<T> lst)
{
    if (lst.isEmpty())
        return accumul;
    else
        return f(lst.car(), foldr(f, accumul, lst.cdr()));
}
template<typename S, typename T, typename F>
S foldl(F f, S accumul, List<T> lst)
{
    if (lst.isEmpty())
        return accumul;
    else
        return foldl(f, f(accumul, lst.car()), lst.cdr());
}
template<typename T, typename F>
void foreach(F f, List<T> lst)
{
    if (!lst.isEmpty()) {
        f(lst.car());
        foreach(f, lst.cdr());
    }
}
template<class T>
void print(List<T> lst)
{
    foreach([](T v)
    {
        cout << v << "->";
    }, lst);
    cout << ".";
}

template<typename T>
List<T> concat(const List<T> a, const List<T> b)
{
    if (a.isEmpty())
        return b;
    return List<T>(a.car(), concat(a.cdr(), b));
}
template<typename T>
T findmax(const List<T>  a)
{
    return foldl([](T acc, T x){return (acc>x) ? acc : x;}, a.car(), a);
}


List<int> qqsort(const List<int> a)
{

    if (a.isEmpty()) return a;

    int piv = a.car();

    List<int> le = qqsort(filter<int>([=](int x){return piv>x;}, a.cdr()));
    List<int> ri = qqsort(filter<int>([=](int x){return piv<=x;}, a.cdr()));
    return concat<int>(le, cons(piv, ri));
}

int main()
{
    List<int> l;
    List<int> n = cons(4, cons(5, cons(6, cons(7, cons<int>(8, l)))));
    l.show();
    n.show();

    auto m0 = mapl<double>([](int i){return i*0.5;} , n); // List<double>
    m0.show();
    n.show();
    auto m = mapl<int>([](int i){return i+1;} , n); // List<int>
    m.show();

    List<string> s = cons("abc"s, cons("defg"s, cons("hijkl"s, List<string>())));
    s.show();

    auto s_len = mapl<size_t>([](string r){return r.size();}, s);
    s_len.show();


    m = filter<int>([](int i){ return i%2==0;}, n);
    m.show();

    s = filter<string>([](string r){ return r.size()>4;}, s);
    s.show();


    cout << foldl([](int acc, int i){return acc+i;}, 0, m) << endl;
    print(m); // or print<int>(m);
    cout << endl;
    List<List<int>> ll;
    auto mm = cons(m, cons(n, ll)); // List<List<int>>
    foreach([](List<int> l) { cout << "["; print(l); cout << "]" << "->";}, mm);
    cout << endl;

    auto g = concat(n, m); //List<int>
    g.show();
    cout << findmax(g) << endl;

    List<int> ns = cons(9, cons(10, cons(6, cons(5, cons(4, cons(8, cons(7, l)))))));
    cout << "Before: ";
    ns.show();
    ns = qqsort(ns);
    cout << "After: ";
    ns.show();

}


Overwriting E10_02.cpp


In [ ]:
%%shell
g++ E10_02.cpp -std=c++1z -o E10_02
./E10_02

.
4->5->6->7->8->.
2->2.5->3->3.5->4->.
4->5->6->7->8->.
5->6->7->8->9->.
abc->defg->hijkl->.
3->4->5->.
4->6->8->.
hijkl->.
18
4->6->8->.
[4->6->8->.]->[4->5->6->7->8->.]->
4->5->6->7->8->4->6->8->.
8
Before: 9->10->6->5->4->8->7->.
After: 4->5->6->7->8->9->10->.


延伸閱讀：C++17 fold expression

https://baptiste-wicht.com/posts/2015/05/cpp17-fold-expressions.html



延伸閱讀：關於 explicit，copy-initialization 和 direct-initialization 的說明

http://en.cppreference.com/w/cpp/language/explicit



延伸閱讀：關於 lambda function

http://en.cppreference.com/w/cpp/language/lambda

## 實作 Integer Vectors
### 介紹 initializer list, move constructor, move assignment


```c++
#include <iostream>
#include <initializer_list>
#include <algorithm>
class IntVec {
};
```

我們用一個常見的範例來當作練習:實作能夠動態增加元素的一維陣列 (vector)。為了方便解釋，我們只允許 vector 裡面放整數。我們把打算自訂的 class 取名為 IntVec，它需要三個 private members：
```c++
private:
    unsigned int size_;
    unsigned int capacity_;
    int* elem_;
```

其中 `size_` 是用來記住 vector 中有實際上多少個元素， `capacity_` 則是目前 vector 可以容納的元素上限，另外 `elem_` 則是指標，用來指向實際存放整數的陣列的開頭位址。

接下來就是定義所們需要的 public member functions。




#### Constructors
首先是各種不同的 constructors。

##### default constructor
我們要的 default constructor 應該會長得像底下這樣
```c++
    IntVec() : size{0}_, capacity_{8}, elem_{new int [capacity_]} {}
```
直接設定每個 member 的預設值，`size_` 是 0 表示 vector 裡面沒有任何元素， `capacity_` 是 8 則是預設最多能放 8 個整數，等到快放滿的時候，我們再動態調整大小。而 `elem_` 這個指標，預設會指向一個包含 8 個整數的連續空間，我們用
`new int [需要的數量]` 來取得能夠放置指定數量的連續空間。
這個 constructor 的主體不需要做其他的事情，所以只有 `{ }`
當我們在主程式裡面寫

```c++
	IntVec v;
```
就會用上面的 default constructor 來產生物件。

注意，在這個例子裡， class 裡面 member 宣告的順序不能調換成底下這樣，

```c++
private:
    unsigned int size_;
    int* elem_;
    unsigned int capacity_;
```

因為上面的初始設定中 `elem_` 會用到 `capacity_` 所以宣告順序必須先是

```c++
    unsigned int capacity_;
    int* elem_;
```

在 class 建構的時候，會依照 member 宣告的順序來產生他們的內容，而不是照他們在 initializer list 的順序，例如

```c++
IntVec() : size{0}_, elem_{new int [capacity_]}, capacity_{8}  {}
```

這裡即使調換順序也不會造成影響。



### copy constructor

```c++
    IntVec(const IntVec& w) : size_{w.size_}, capacity_{w.capacity_},
        elem_{new int [w.capacity_]}
    {
        std::cout << "copy constructor\n";
        for (unsigned int i=0; i<w._size; i++)
            elem_[i] = w.elem_[i];
    }
```
當我們再產生一個新的 vector 的時候，如果希望直接複製一個既有的 vector 的內容，就會用到 copy constructor。例如底下的用法
```c++
	IntVec u{v};
```
或是
```c++
	IntVec u = v;
```
所以 copy constructor 要做的事情，首先是設定對應的 members，包括 `size_` 和 `capacity_` 以及指標 `elem_`。我們一樣要先取得足夠的空間，然後用 `elem_` 記住該空間的啟始位址，再來就要把 `elem_` 的內容從傳入的 IntVect 複製過來。上面的 copy constructor 函數裡的 `for` 迴圈，就是為了把傳進來的參數 `w` 的 `elem_` 逐一複製到自己的 `elem_`。




##### initializer list constructor
我們也希望能使用底下的寫法來產生新的 vector
```c++
	IntVec v{1,2,3,4};
```
要達到這個效果，我們必須定義 constructor 能夠接收 initializer list 當作參數。
```c++
    IntVec(std::initializer_list<int> lst):
        size_{lst.size()}
    {
        capacity_ = (lst.size()*2>8) ? lst.size()*2 : 8;
        elem_ = new int [capacity_];
        if (lst.size()>0)
            std::copy(lst.begin(), lst.end(), elem_);
    }
```
這要用到 ``` std::initializer_list<T>``` 來達成，所以我們在程式碼的最前面加了
`#include <initializer_list>`。
傳入的參數 lst 是一個由整數構成的 initializer list，當我們在程式碼裡面寫類似 `{1,2,3,4}` 這樣的東西，就會被解讀為 initializer list 然後做對應的處理。我們可以用 `.size()` 函數來取得 initializer list 的大小，然後用 `.begin()` 和 `.end()` 來取得第一個元素和最後一個元素的位址，如此一來就可以用 `std::copy` 函數，來將 initializer list 的內容複製到我們自訂的 vector 的 `elem` 指標所指到的那塊空間中。


##### 直接指定要產生的 vector 的大小
```c++
    IntVec(unsigned int sz):
        _size{sz},
        _capacity{2*sz},
        elem{new int [2*sz]} { std::cout << "growing...\n"; }

```
我們先取得兩倍大的空間，預備讓後續新增的元素可以直接放入，不需要每次都調整 `elem` 的大小。
有一個小地方要注意，當我們在程式碼裡面寫
```c++
	IntVec t{30};
```
會使用 initializer list 對應的 constructor，產生一個 vector 裡面只放了一個元素 `30`。如果是
```c++
	IntVec t(30);
```
才會是產生一個能夠放至少 30 個整數的 vector。

#### Desctructor
物件消失時要把 elem 占用的空間還回去。
```c++
    ~IntVec() {
        delete [] elem;
    }
```

#### Operators
我們還要自訂 operators。例如為了讓 `IntVec` 也能像一般的 array 使用方括號`[]`來取得元素的內容，我們在 class 裡面加入
```c++
    int & operator[](int i) {
    	// no boundary check
        return elem[i];
    }
```
這樣一來，如果程式碼裡面寫
```c++
	IntVec v{5,10,15,20};
    std::cout << v[2] << std::endl;
    v[2] = 16;
    std::cout << v[2] << std::endl;
```
就可以在螢幕上顯示出 v 的編號`2` 的元素 `15`，或是把值改成 `16`。

##### copy assignment
```c++
  IntVec& operator=(const IntVec& w)
    {
        IntVec v{w};
        std::cout << "copy assignment\n";
        std::swap(elem, v.elem);
        _size = v._size;
        _capacity = v._capacity;
        return *this;
    }
```
我們也重新定義 `=`，這樣才能夠在程式碼裡面寫
```c++
	IntVec s{1,2,3,4};
  IntVec t{5,6,7};
  t = s;
```
上面的程式用到了一些慣用的寫法。首先在函數中產生一個 local object `v`，`IntVec v{w};`，它的內容直接從 `w` 複製過來，當這個函數結束的時候，`v`會被清除掉 (因為有 destructor)。接下來我們把 `elem` 和 `v.elem` 兩個指標交換，如此一來，自已 (this) 就會取得和 `w` 完全相同的 `elem` 內容，但 `v` 的 `elem` 則會變成是 this 原本的 `elem` 內容，最後當 `v` 被清除的時候，就會一併清除原本 this 所擁有的 `elem`。

##### 重新定義 operator<<
這個 operator 不是 IntVec 的 member function，只是為了顯示
```c++
std::ostream& operator<<(std::ostream& os, IntVec& v)
{
    os << "[";
    for (unsigned int i=0; i<v._size; i++)
        os << v[i] << ", ";
    os << "]";
    return os;
}
```
所以我們在 class 裡面列為 friend，這樣才能使用 `._size`。
```c++
	friend std::ostream& operator<<(std::ostream& os, IntVec& v);
```

##### 模仿實作 push_back 函數
我們希望也能用 `push_back` 在 vector 的最後加入新的資料。
```c++
    void push_back(const int val) {
        if (_size+1 >= _capacity) {
            IntVec tmp(_capacity);
            _capacity = tmp._capacity;
            for (unsigned int i=0; i<_size; i++)
                tmp.elem[i] = elem[i];
            std::swap(elem, tmp.elem);
        }
        elem[_size] = val;
        ++_size;
    }
```
正常情況只需要直接在 `elem[_size]` 放入新的資料，並且`++_size;`。
如果 `_capacity` 不夠放新的資料，就要先產生一個新的足夠的空間 (用 local object 方式)，把舊的資料複製過去，然後 swap，這樣當 local object 被清除時，就會一併清掉舊的資料。



#### 與 rvalue reference 有關的兩個 member functions

##### move constructor

```c++
    IntVec(IntVec&& w): _size{w._size}, _capacity{w._capacity}, elem{w.elem}
    {
        std::cout << "move constructor\n";
        w._size = 0;
        w._capacity = 0;
        w.elem = nullptr;
    }
```

##### move assignment
```c++
    IntVec& operator=(IntVec&& w) {
        std::cout << "move assignment\n";
        if (this != &w) {
            delete [] elem;

            elem = w.elem;
            _size = w._size;
            _capacity = w._capacity;

            w.elem = nullptr;
            w._size = 0;
            w._capacity = 0;
        }
        return *this;
    }
```

以 move assignment 為例，參數的 type 是 `IntVec&&`，兩個 ref 符號，表示它接收的是 rvalue reference，和 copy assignment 的參數 `IntVec&` 不同 (只有一個 ref 符號，是 lvalue reference)。所謂的 lvalue，是指可以放在等號左邊的東西，例如 `int a = 3;` 其中的整數變數 a 就是可以放在等號左邊的東西，而相對地，3 是不能放在等號左邊的東西 (我們不會寫 `3 = 5;` 或 `3 = x;`)，我們稱作 rvalue。引用 Bjarne Stroustrup "The C++ Programming Language (4th Edition)" 書中的話

>> an rvalue reference is a reference to something that nobody else can assign to, so that we can safely “steal” its value.

因為我們有實際寫出 move assignment 版本的 `operator=`，所以如果程式碼有用到
```c++
	IntVec w, v;
  w = {2,3,4};
```
等號右邊的 `{2,3,4}` 是 rvalue，所以可以對應到 rvalue reference，因此 move assignment 版本的 `operator=` 會被呼叫。這時候發生的事情是，先用 initializer list 產生一個暫時的物件，然後 move assignment 會把那個暫時的物件的內容搬到 `w`，假設我們沒有實作 move assignment，則上面的式子只好用 copy assignment 來達成，這時候發生的事情則會是，先用 initializer list 產生一個暫時的物件，然後用 copy assignment 來複製，過程中還會用 copy constructor 先產生一個臨時的副本，讓 w 可以得到我們想複製的內容。所以，這種情況下，有實際做出 move assignment 可以省掉產生臨時副本。

我們可以把上面的 move constructor 和 move assignment 範例當成慣用寫法學起來，套用到其他自訂的 class。

## 完整的程式碼


In [ ]:
%%writefile E12_01.cpp
#include <iostream>
#include <initializer_list>
#include <algorithm>
class IntVec {
public:
    IntVec() :
        _size{0},
        _capacity{8},
        elem{new int [_capacity]} { std::cout<<"Default constructor\n";}

    IntVec(const IntVec& w) : _size{w._size}, _capacity{w._capacity},
        elem{new int [w._capacity]}
    {
        std::cout << "copy constructor\n";
        for (unsigned int i=0; i<w._size; i++)
            elem[i] = w.elem[i];
    }

    IntVec(std::initializer_list<int> lst):
        _size{lst.size()}
    {
        _capacity = (lst.size()*2>8) ? lst.size()*2 : 8;
        elem = new int [_capacity];
        if (lst.size()>0)
            std::copy(lst.begin(), lst.end(), elem);
    }

    IntVec(unsigned int sz):
        _size{sz},
        _capacity{2*sz},
        elem{new int [2*sz]} { std::cout << "growing...\n"; }


    ~IntVec() {
        delete [] elem;
    }

    int & operator[](int i) {
        // no boundary check
        return elem[i];
    }


    IntVec& operator=(const IntVec& w)
    {
        IntVec v{w};
        std::cout << "copy assignment\n";
        std::swap(elem, v.elem);
        _size = v._size;
        _capacity = v._capacity;
        return *this;
    }

    // rvalue reference
    IntVec(IntVec&& w): _size{w._size}, _capacity{w._capacity}, elem{w.elem}
    {
        std::cout << "move constructor\n";
        w._size = 0;
        w._capacity = 0;
        w.elem = nullptr;
    }

    // rvalue reference
    IntVec& operator=(IntVec&& w) {
        std::cout << "move assignment\n";
        if (this != &w) {
            delete [] elem;

            elem = w.elem;
            _size = w._size;
            _capacity = w._capacity;

            w.elem = nullptr;
            w._size = 0;
            w._capacity = 0;
        }
        return *this;
    }


    void push_back(const int val) {
        if (_size+1 >= _capacity) {
            IntVec tmp(_capacity);
            _capacity = tmp._capacity;
            for (unsigned int i=0; i<_size; i++)
                tmp.elem[i] = elem[i];
            std::swap(elem, tmp.elem);
        }
        elem[_size] = val;
        ++_size;
    }

    friend std::ostream& operator<<(std::ostream& os, IntVec& v);

private:
    unsigned int _size;
    unsigned int _capacity;
    int* elem;
};

std::ostream& operator<<(std::ostream& os, IntVec& v)
{
    os << "[";
    for (unsigned int i=0; i<v._size; i++)
        os << v[i] << ", ";
    os << "]";
    return os;
}

int main()
{
    IntVec t;
    IntVec v{1,2,3};
    std::cout << v << std::endl;
    for (int e; std::cin>>e;)
        v.push_back(e);
    std::cout << v << std::endl;
    IntVec w{v};
    std::cout << w << std::endl;
    w = {3,4,5};
    std::cout << w << std::endl;
    IntVec u = std::move(w);
    u[2] = 6;
    std::cout << u << std::endl;
    std::cout << w << std::endl;
}